In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [136]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

In [137]:
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [138]:
if torch.cuda.is_available():
    avDev = torch.device("cuda")
else:
    avDev = torch.device("cpu")

print(avDev)

cuda


In [139]:
torch.manual_seed(1)
np.random.seed(1)

In [140]:
len(trainset)

50000

In [141]:
# STEP 2: MAKING DATASET ITERABLE
 
batch_size = 500
n_iters = 5000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
 
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=len(testset), 
                                          shuffle=False)
 

In [142]:
'''
STEP 3: CREATE MODEL CLASS
'''
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim, activation_fn):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, 3000)
        self.linear1_drop = nn.Dropout(0.2)
        self.linear2 = nn.Linear(3000, 3000)
        self.linear2_drop = nn.Dropout(0.2)
        self.linear3 = nn.Linear(3000, output_dim)
        self.activation_fn = activation_fn
     
    def forward(self, x):
        out = self.activation_fn(self.linear1(x))
        layer1_out = self.linear1_drop(out)
        out = self.activation_fn(self.linear2(layer1_out))
        layer2_out = self.linear2_drop(out)
        out = self.linear3(layer2_out)
        return out
    

In [143]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 3*32*32
output_dim = 10
 
model = LogisticRegressionModel(input_dim, output_dim, nn.ReLU())

In [144]:
#  USE GPU FOR MODEL  #
#######################
 
model.to(avDev)
 
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss().to(avDev)


In [145]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.03
 
optimizer_SGD = torch.optim.SGD(model.parameters(), lr=learning_rate)

optimizer_Adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

optimizer_Adagrad = torch.optim.Adagrad(model.parameters(), lr=learning_rate)

optimizer_Adadelta = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

optimizer_RMSprop = torch.optim.RMSprop(model.parameters(), lr=learning_rate)


In [146]:
print(model)

LogisticRegressionModel(
  (linear1): Linear(in_features=3072, out_features=3000, bias=True)
  (linear1_drop): Dropout(p=0.2, inplace=False)
  (linear2): Linear(in_features=3000, out_features=3000, bias=True)
  (linear2_drop): Dropout(p=0.2, inplace=False)
  (linear3): Linear(in_features=3000, out_features=10, bias=True)
  (activation_fn): ReLU()
)


In [147]:
#L1, and L2 regularization parms

lambda1 = 0.001
lambda2 = 0.01

In [148]:
'''
STEP 7: TRAIN THE MODEL
'''
def train_model(optimizer):
    loss_save = np.empty(n_iters-1)

    iter = 0
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):

            images = images.view(-1, 3*32*32).to(avDev)
            labels = labels.to(avDev)

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()


            # Forward pass to get output/logits
            outputs = model(images)
    #         outputs = model(images)

            # Calculate Loss: softmax --> cross entropy loss
            cross_entropy_loss = criterion(outputs, labels)#


            all_linear1_params = torch.cat([x.view(-1) for x in model.linear1.parameters()])
            all_linear2_params = torch.cat([x.view(-1) for x in model.linear2.parameters()])
            l1_regularization = lambda1 * torch.norm(all_linear1_params, 1)
            l2_regularization = lambda2 * torch.norm(all_linear2_params, 2)

#             loss = cross_entropy_loss + l1_regularization + l2_regularization

            loss = cross_entropy_loss + l2_regularization
            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            #Save Loss    

            loss_save[iter-1] = loss.item()
            iter += 1

            if iter % 200 == 0:
                # Calculate Accuracy         
                correct = 0
                total = 0
                # Iterate through test dataset
                for images, labels in test_loader:
                    #######################
                    #  USE GPU FOR MODEL  #
                    #######################
                    images = images.view(-1, 3*32*32).to(avDev)

                    # Forward pass only to get logits/output
                    outputs = model(images)

                    # Get predictions from the maximum value
                    _, predicted = torch.max(outputs.data, 1)

                    # Total number of labels
                    total += labels.size(0)

                    #######################
                    #  USE GPU FOR MODEL  #
                    #######################
                    # Total correct predictions
                    correct += (predicted.cpu() == labels.cpu()).sum().float()

                accuracy = 100. * correct / total

                # Print Loss
                print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.ReLU())
model.to(avDev)
train_model(optimizer_SGD)

Iteration: 200. Loss: 2.626703977584839. Accuracy: 10.449999809265137
Iteration: 400. Loss: 2.6186680793762207. Accuracy: 10.380000114440918


In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Tanh())
model.to(avDev)
train_model(optimizer_SGD)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Sigmoid())
model.to(avDev)
train_model(optimizer_SGD)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.ReLU())
model.to(avDev)
train_model(optimizer_Adam)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Tanh())
model.to(avDev)
train_model(optimizer_Adam)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Sigmoid())
model.to(avDev)
train_model(optimizer_Adam)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.ReLU())
model.to(avDev)
train_model(optimizer_Adagrad)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Tanh())
model.to(avDev)
train_model(optimizer_Adagrad)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Sigmoid())
model.to(avDev)
train_model(optimizer_Adagrad)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.ReLU())
model.to(avDev)
train_model(optimizer_Adadelta)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Tanh())
model.to(avDev)
train_model(optimizer_Adadelta)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Sigmoid())
model.to(avDev)
train_model(optimizer_Adadelta)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.ReLU())
model.to(avDev)
train_model(optimizer_RMSprop)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Tanh())
model.to(avDev)
train_model(optimizer_RMSprop)

In [ ]:
model = LogisticRegressionModel(input_dim, output_dim, nn.Sigmoid())
model.to(avDev)
train_model(optimizer_RMSprop)

In [ ]:
correct = 0
total = 0
model.to(avDev)

with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.view(-1, 3*32*32).to(avDev)
        labels = labels.to(avDev)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the  train images: %d %%' % (
    100 * correct / total))

In [ ]:
print(images.size())

In [ ]:
outputs.size()

In [ ]:
labels.size()

# Learning Curve

In [ ]:
ax = sns.lineplot(data = loss_save)
ax.set(xlabel = "Iterations", ylabel="Loss")
plt.show()

# Confusion Matrix

In [ ]:
model.to(avDev)

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.view(-1, 3*32*32).to(avDev)
        labels = labels.to(avDev)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
cm = confusion_matrix(predicted.cpu(), labels.cpu()).astype(np.int)

In [ ]:
import pandas as pd
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
plt.figure(figsize=(12,10))
df_cm = pd.DataFrame(cm,CLASSES,CLASSES)
sns.set(font_scale=1.4)
ax = sns.heatmap(df_cm,annot=True,fmt='d')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()